## CS6360 Programming Assignment 2: Bayesian Networks
The python notebook for the CS6360 programming assignment 2 completed and coded by Anne Tumlin.

### Part 1: Implementing the Variable Elimination Algorithm

Import and install all necessary files. 

In [58]:
import numpy as np
from IPython.display import Image, display
import pydotplus

Implement factor class. 

In [59]:
class Factor:
    def __init__(self, variables, values):
        self.variables = variables  # A list of variable names ['A', 'B', 'C', ...]
        self.values = values  # A numpy array of values where rows correspond to different states
        
    def __repr__(self):
        return f"Factor(variables={self.variables}, values=\n{self.values})"

Example implementation of factor and factor list for testing. 

In [60]:
# Bayesian network: A -> C <- B

# Factor for P(A)
factor_A = Factor(variables=['A'], 
                  values=np.array([
                      [True, 0.6],
                      [False, 0.4]
                  ]))

# Factor for P(B)
factor_B = Factor(variables=['B'], 
                  values=np.array([
                      [True, 0.7],
                      [False, 0.3]
                  ]))

# Factor for P(C|A, B)
factor_C_given_A_B = Factor(variables=['A', 'B', 'C'], 
                            values=np.array([
                                [True,  True,  True,  0.9],
                                [True,  True,  False, 0.1],
                                [True,  False, True,  0.5],
                                [True,  False, False, 0.5],
                                [False, True,  True,  0.8],
                                [False, True,  False, 0.2],
                                [False, False, True,  0.3],
                                [False, False, False, 0.7]
                            ]))

# Factor list
factorList = [factor_A, factor_B, factor_C_given_A_B]
print(f'Factor A: \n {factor_A}')
print(f'Factor B: \n {factor_B}')
print(f'Factor C: \n {factor_C_given_A_B}')


Factor A: 
 Factor(variables=['A'], values=
[[1.  0.6]
 [0.  0.4]])
Factor B: 
 Factor(variables=['B'], values=
[[1.  0.7]
 [0.  0.3]])
Factor C: 
 Factor(variables=['A', 'B', 'C'], values=
[[1.  1.  1.  0.9]
 [1.  1.  0.  0.1]
 [1.  0.  1.  0.5]
 [1.  0.  0.  0.5]
 [0.  1.  1.  0.8]
 [0.  1.  0.  0.2]
 [0.  0.  1.  0.3]
 [0.  0.  0.  0.7]])


Implement the restrict function.

In [61]:
# Function that restricts a variable to some value in a given factor
def restrict(factor, variable, value):
    if variable not in factor.variables:
        return factor  # The variable is not in the factor, return the factor unchanged.
        
    var_index = factor.variables.index(variable)
    restricted_values = factor.values[factor.values[:, var_index] == value]
    
    new_variables = factor.variables 
    new_values = restricted_values  
    
    return Factor(variables=new_variables, values=new_values)

# Testing restrict function
restricted_f_C_given_A = restrict(factor_C_given_A_B, 'B', True)
print(restricted_f_C_given_A)

Factor(variables=['A', 'B', 'C'], values=
[[1.  1.  1.  0.9]
 [1.  1.  0.  0.1]
 [0.  1.  1.  0.8]
 [0.  1.  0.  0.2]])


Implement the multiply function.

In [62]:
# Function that multiplies two factors
def multiply(factor1, factor2):
    # Find the variables common to both factors
    common_vars = list(set(factor1.variables) & set(factor2.variables))
    # Find the variables that are not common
    vars2 = [var for var in factor2.variables if var not in common_vars]
    
    new_variables = factor1.variables + vars2 
    new_rows = [] 
    
    for row1 in factor1.values:
        for row2 in factor2.values:
            if all(row1[factor1.variables.index(var)] == row2[factor2.variables.index(var)] for var in common_vars):
                # Join the rows on the common variables, and then multiply the probabilities
                new_row = list(row1[:-1]) + [row2[factor2.variables.index(var)] for var in vars2] + [row1[-1] * row2[-1]]
                new_rows.append(new_row)
                
    return Factor(new_variables, np.array(new_rows))

# Testing multiply function
product_factor = multiply(factor_A, factor_B)
print(product_factor)

Factor(variables=['A', 'B'], values=
[[1.   1.   0.42]
 [1.   0.   0.18]
 [0.   1.   0.28]
 [0.   0.   0.12]])


Implement the sumout function.

In [63]:
# function that sums out a variable in a given factor.
def sumout(factor, variable):
    if variable not in factor.variables:
        return factor # The variable is not in the factor, return the factor unchanged.

    new_variables = [v for v in factor.variables if v != variable]

    sum_dict = {}

    for row in factor.values:
        key = tuple(row[i] for i in range(len(factor.variables)) if factor.variables[i] != variable)

        if key not in sum_dict:
            sum_dict[key] = 0
        
        sum_dict[key] += row[-1]
    
    new_values_list = [[*key, prob] for key, prob in sum_dict.items()]
    new_values = np.array(new_values_list, dtype=object)

    for i, row in enumerate(new_values):
        new_values[i][-1] = float(row[-1])

    return Factor(new_variables, new_values)

# Testing sumout function
result_factor = sumout(factor_C_given_A_B, 'B')
print(result_factor)

Factor(variables=['A', 'C'], values=
[[1.0 1.0 1.4]
 [1.0 0.0 0.6]
 [0.0 1.0 1.1]
 [0.0 0.0 0.8999999999999999]])


Implement the normalize function.

In [64]:
# function that normalizes a factor by dividing each entry by the sum of all the entries. 
def normalize(factor):
    total_prob = np.sum(factor.values[:, -1])
    if total_prob > 0:
        factor.values[:, -1] /= total_prob 
    return factor

# Testing normalize function
normalized_result_factor = normalize(result_factor)
print(normalized_result_factor)

Factor(variables=['A', 'C'], values=
[[1.0 1.0 0.35]
 [1.0 0.0 0.15]
 [0.0 1.0 0.275]
 [0.0 0.0 0.22499999999999998]])


Implement the inference function.

In [65]:
# function that computes 𝑷(querryList|evidenceList) by variable elimination.
def inference(factorList, queryVariables, orderedListOfHiddenVariables, evidenceList):
    
    # Restrict the factors in factorList according to the evidence in evidenceList
    for evidence_var, evidence_value in evidenceList.items():
        factorList = [restrict(factor, evidence_var, evidence_value) if evidence_var in factor.variables else factor for factor in factorList]
    
    # Calculate the product of the factors in factorList
    product_factor = factorList[0]
    for factor in factorList[1:]:
        product_factor = multiply(product_factor, factor)
        
    # Sum out the the variables in the order given in orderedListOfHiddenVariables
    for hidden_var in orderedListOfHiddenVariables:
        product_factor = sumout(product_factor, hidden_var)
    
    # Normalize when a probability distribution that sums up to 1 is desired
    result_factor = normalize(product_factor)
    
    if queryVariables:
        # Filter the values to include only query variable columns
        indices_to_keep = [product_factor.variables.index(var) for var in queryVariables]
        filtered_values = product_factor.values[:, indices_to_keep + [-1]]  # Add the probability column
        result_factor = Factor(variables=queryVariables, values=filtered_values)

    return result_factor

# Testing inference function
result_factor = inference(factorList, queryVariables=['A'], orderedListOfHiddenVariables=['B'], evidenceList={'C': True})
print(result_factor)

Factor(variables=['A'], values=
[[1.0 0.6428571428571428]
 [0.0 0.3571428571428571]])


### Part 2: Construct The Bayesian Network
Here is the graphical representation of the bayesian network detailed in the instructions:

![Sample Image](./bayesiannetwork.jpeg)

#### Conditional Probability Tables
P(OC)
| OC           | P(OC)     |
| ------------ | --------- |
| True (owns)  | 0.8       |
| False (none) | 0.2       |

P(Trav)
| Trav       | P(Trav)   |
| ---------- | --------- |
| True       | 0.05      |
| False      | 0.95      |

P(Fraud|Trav)
| Trav       | Fraud     | P(Fraud \| Trav)     |
| ---------- | --------- | -------------------- |
| True       | True      | 0.01                |
| True       | False     | 0.99                |
| False      | True      | 0.004               |
| False      | False     | 0.996               |

P(FP|Fraud,Trav)
| Trav       | Fraud     | FP        | P(FP \| Fraud, Trav)       |
| ---------- | --------- | --------- | -------------------------- |
| True       | True      | True      | 0.9                       |
| True       | True      | False     | 0.1                       |
| True       | False     | True      | 0.9                       |
| True       | False     | False     | 0.1                       |
| False      | True      | True      | 0.1                       |
| False      | True      | False     | 0.9                       |
| False      | False     | True      | 0.01                      |
| False      | False     | False     | 0.99                      |

P(IP|Fraud,OC)
| OC         | Fraud     | IP        | P(IP \| Fraud, OC)       |
| ---------- | --------- | --------- | ------------------------ |
| True       | True      | True      | 0.15                    |
| True       | True      | False     | 0.85                    |
| True       | False     | True      | 0.1                     |
| True       | False     | False     | 0.9                     |
| False      | True      | True      | 0.051                   |
| False      | True      | False     | 0.949                   |
| False      | False     | True      | 0.001                   |
| False      | False     | False     | 0.999                   |

P(CRP|OC)
| OC         | CRP       | P(CRP \| OC)            |
| ---------- | --------- | ----------------------- |
| True       | True      | 0.1                     |
| True       | False     | 0.9                     |
| False      | True      | 0.01                    |
| False      | False     | 0.99                    |

Encoding the conditional probabilities above into factors and a factor list.

In [66]:
# Factor for OC 
factor_OC = Factor(variables=['OC'], 
                   values=np.array([
                       [True, 0.8],
                       [False, 0.2]
                   ]))
                   
# Factor for Trav 
factor_Trav = Factor(variables=['Trav'], 
                     values=np.array([
                         [True, 0.05],
                         [False, 0.95]
                     ]))

# Factor for Fraud given Trav 
factor_Fraud_given_Trav = Factor(variables=['Trav', 'Fraud'], 
                                 values=np.array([
                                     [True,  True,  0.01],
                                     [True,  False, 0.99],
                                     [False, True,  0.004],
                                     [False, False, 0.996]
                                 ]))

# Factor for FP given Fraud and Trav
factor_FP_given_Fraud_Trav = Factor(variables=['Trav', 'Fraud', 'FP'], 
                                    values=np.array([
                                        [True,  True,  True,  1],
                                        [True,  True,  False, 0],
                                        [True,  False, True,  0.9],
                                        [True,  False, False, 0.1],
                                        [False, True,  True,  0.1],
                                        [False, True,  False, 0.9],
                                        [False, False, True,  0.01],
                                        [False, False, False, 0.99]
                                    ]))

# Factor for IP given Fraud and OC
factor_IP_given_Fraud_OC = Factor(variables=['OC', 'Fraud', 'IP'], 
                                  values=np.array([
                                      [True,  True,  True,  0.15],
                                      [True,  True,  False, 0.85],
                                      [True,  False, True,  0.1],
                                      [True,  False, False, 0.9],
                                      [False, True,  True,  0.051],
                                      [False, True,  False, 0.949],
                                      [False, False, True,  0.001],
                                      [False, False, False, 0.999]
                                  ]))

# Factor for CRP given OC
factor_CRP_given_OC = Factor(variables=['OC', 'CRP'], 
                             values=np.array([
                                 [True,  True,  0.1],
                                 [True,  False, 0.9],
                                 [False, True,  0.01],
                                 [False, False, 0.99]
                             ]))

factorList = [
    factor_OC, 
    factor_Trav, 
    factor_Fraud_given_Trav, 
    factor_FP_given_Fraud_Trav, 
    factor_IP_given_Fraud_OC, 
    factor_CRP_given_OC
]

### Part 2(b)(i)
What is the prior probability (i.e., before we search for previous computer related purchases
and before we verify whether it is a foreign and/or an internet purchase) that the current
transaction is a fraud?

**Find: P(Fraud)**

In [67]:
# Calculate the prior probability for 'Fraud' by marginalizing out all variables according to the order provided
prior_fraud = inference(factorList=factorList,
                        queryVariables=['Fraud'],
                        orderedListOfHiddenVariables=['Trav', 'FP', 'IP', 'OC', 'CRP'],
                        evidenceList={})

print(prior_fraud)

Factor(variables=['Fraud'], values=
[[1.0 0.004299999999999999]
 [0.0 0.9956999999999999]])


**ANSWER**: The prior probability that the current transaction is a fraud is **0.429%**.

### Part 2(b)(ii)
What is the probability that the current transaction is a fraud once we have verified that it is a
foreign transaction, but not an internet purchase and that the card holder purchased computer
related accessories in the past week?

**Find: P(Fraud|FP=T,IP=F,CRP=T)**

In [68]:
# Calculate the conditional probability of fraud given the evidence
conditional_fraud_prob = inference(factorList=factorList,
                                   queryVariables=['Fraud'],
                                   orderedListOfHiddenVariables=['Trav', 'OC'],
                                   evidenceList={'FP': True, 'IP': False, 'CRP': True})

print(conditional_fraud_prob)

Factor(variables=['Fraud'], values=
[[1.0 0.015156688340694066]
 [0.0 0.9848433116593059]])


**ANSWER**: The prior probability that the current transaction is a fraud given it is a foreign transaction, not an internet purchase, and the user purchased computer related accessories in the past week is **1.516%**.

### Part 2(b)(iii)
After computing those probabilities, the fraud detection system raises a flag and recommends
that the card holder be called to confirm the transaction. An agent calls at the domicile of the
card holder but she is not home. Her spouse confirms that she is currently out of town on a
business trip. How does the probability of a fraud changes based on this new piece of
information?

**Find: P(Fraud|FP=T,IP=F,CRP=T,Trav=T)**

In [69]:
# Calculate the conditional probability of fraud given the new evidence
updated_conditional_fraud_prob = inference(factorList=factorList,
                                           queryVariables=['Fraud'],
                                           orderedListOfHiddenVariables=['OC'] ,
                                           evidenceList={'FP': True,'IP': False,'CRP': True,'Trav': True})

print(updated_conditional_fraud_prob)

Factor(variables=['Fraud'], values=
[[1.0 0.010490281144277188]
 [0.0 0.9895097188557229]])


**ANSWER**: The prior probability that the current transaction is a fraud given it is a foreign transaction, not an internet purchase, the user purchased computer related accessories in the past week, and is traveling is **1.049%**.

### Part 2(b)(iv)
Suppose you are not a very honest employee and you just stole a credit card. You know that
the fraud detection system uses the Bayes net designed earlier, but you still want to make an
important purchase over the internet. What action you can take prior to your internet purchase
to reduce the risk that the transaction will be rejected as a possible fraud?. Indicate by how
much the probability of a fraud gets reduced by taken this action.

**ANSWER:** To lower the risk of being flagged as fraudulent when making a large purchase online, you can make a computer-related purchase in the week prior. This is because users who make such purchases are generally considered more trustworthy due to the conditional probabilities assigned to them. As they are more likely to own a computer, their purchases are deemed less likely to be fraud when making an internet purchase.

In [70]:
# Calculate the probability of fraud given it's an internet purchase without a prior computer-related purchase
prob_fraud_no_crp = inference(factorList,
                              queryVariables=['Fraud'],
                              orderedListOfHiddenVariables=['Trav', 'OC', 'FP', 'CRP'],
                              evidenceList={'IP': True, 'CRP': False})


# Calculate the probability of fraud given it's an internet purchase with a prior computer-related purchase
prob_fraud_with_crp = inference(factorList,
                                queryVariables=['Fraud'],
                                orderedListOfHiddenVariables=['Trav', 'OC', 'FP'],
                                evidenceList={'IP': True, 'CRP': True})


# The reduction in fraud probability by making the computer-related purchase
prob_reduction = prob_fraud_no_crp.values[:, -1] - prob_fraud_with_crp.values[:, -1]

print("Probability of Fraud without CRP:", prob_fraud_no_crp)
print("Probability of Fraud with CRP:", prob_fraud_with_crp)
print("Reduction in Probability of Fraud:", prob_reduction)

Probability of Fraud without CRP: Factor(variables=['Fraud'], values=
[[1.0 0.0070145563176208636]
 [0.0 0.9929854436823792]])
Probability of Fraud with CRP: Factor(variables=['Fraud'], values=
[[1.0 0.0064889028546204755]
 [0.0 0.9935110971453796]])
Reduction in Probability of Fraud: [0.0005256534630003881 -0.0005256534630003751]


**Additional Question:** Discuss how easy/difficult it may be to scale up Bayes net
construction and execution to answer queries.

**Answer:** Scaling up Bayesian Networks for query answering can be a difficult task due to the exponential increase in complexity as the number of variables and their interdependencies grows. In domains where strong conditional independence exists and data is available, constructing and using larger networks is easier. However, in domains with dense connectivity and large conditional probability tables, computational difficulties arise. Exact inference is often infeasible for very large networks, making approximate methods necessary. The ease or difficulty of scaling up depends heavily on the characteristics of the domain, the chosen inference algorithms, and the quality of data used to learn the network's structure and parameters.